Datos sacados de este dataset:
https://www.kaggle.com/paultimothymooney/covid19-containment-and-mitigation-measures

In [1]:
import re
import numpy as np
import pandas as pd

df = pd.read_csv("input/covid_19_containment_measures_data.csv")

print("Setup completado!")

Setup completado!


# Análisis Exploratorio

In [2]:
filas, columnas = df.shape
print(f"El DataFrame esta compuesto de {filas} filas y {columnas} columnas.")

El DataFrame esta compuesto de 1703 filas y 16 columnas.


In [3]:
display(df.head())

,ID,Applies To,Country,Date Start,Date end intended,Description of measure implemented,Exceptions,Implementing City,Implementing State/Province,Keywords,Quantity,Source,Target city,Target country,Target region,Target state
0,163,NaN,Austria,"Mar 16, 2020",NaN,On 10 March 2020 government announced that all...,NaN,NaN,NaN,university closure,NaN,https://en.wikipedia.org/wiki/2020_coronavirus...,NaN,NaN,NaN,NaN
1,132,NaN,Germany,"Feb 01, 2020",NaN,"102 German citizens and 26 relatives, all of w...",NaN,NaN,NaN,international traveller quarantine - risk coun...,0.0001,https://www.tagesschau.de/inland/coronavirus-g...,NaN,NaN,NaN,NaN
2,578,NaN,United Kingdom,"Mar 20, 2020",NaN,"All schools, nurseries and colleges closed.",NaN,NaN,NaN,"nursery school closure, school closure, univer...",NaN,https://www.theguardian.com/world/2020/mar/18/...,NaN,NaN,NaN,NaN
3,372,NaN,United Kingdom,"Mar 16, 2020",NaN,If one person in any household has a persisten...,NaN,NaN,NaN,blanket isolation - symptoms,NaN,https://www.bbc.com/news/uk-51920444,NaN,NaN,NaN,NaN
4,357,NaN,United Kingdom,"Mar 16, 2020",NaN,"By the weekend, those with the most serious he...",NaN,NaN,NaN,isolation advice to elderly,NaN,https://www.bbc.com/news/uk-51920444,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1703 entries, 0 to 1702
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  820 non-null    object 
 1   Applies To                          29 non-null     object 
 2   Country                             1675 non-null   object 
 3   Date Start                          1639 non-null   object 
 4   Date end intended                   242 non-null    object 
 5   Description of measure implemented  1640 non-null   object 
 6   Exceptions                          41 non-null     object 
 7   Implementing City                   127 non-null    object 
 8   Implementing State/Province         179 non-null    object 
 9   Keywords                            1615 non-null   object 
 10  Quantity                            302 non-null    float64
 11  Source                              1517 no

In [5]:
print(df.columns)

Index(['ID', 'Applies To', 'Country', 'Date Start', 'Date end intended',
       'Description of measure implemented', 'Exceptions', 'Implementing City',
       'Implementing State/Province', 'Keywords', 'Quantity', 'Source',
       'Target city', 'Target country', 'Target region', 'Target state'],
      dtype='object')


# Limpieza de datos

In [6]:
# INVESTIGAR NULOS

# Voy a utilizar mucho el campo "Country" por eso voy a analizar los nulos.

# ¿Cuantas entradas tienen el campo "Country" nulo?
print(df["Country"].isna().sum())

# Muestro esas filas:
display(df[df["Country"].isna()].head())

# Como vemos esas filas tienen muchos campos nulos y aportan poca información. 
# Decido borrarlas.
df = df.dropna(subset=['Country'])

28


,ID,Applies To,Country,Date Start,Date end intended,Description of measure implemented,Exceptions,Implementing City,Implementing State/Province,Keywords,Quantity,Source,Target city,Target country,Target region,Target state
1357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1421,NaN,NaN,NaN,"Mar 25, 2020",NaN,A direct 24-hour hotline was set up,NaN,NaN,NaN,phone line,1.0,https://www.moroccoworldnews.com/2020/03/29759...,NaN,NaN,NaN,NaN
1422,NaN,NaN,NaN,"Mar 24, 2020",NaN,Suspension of all inter-city buses,NaN,NaN,NaN,domestic travel limitation,NaN,https://www.moroccoworldnews.com/2020/03/29700...,NaN,NaN,NaN,NaN


In [7]:
# INVESTIGAR DUPLICADOS:

# ¿Hay duplicados?
print(df.duplicated().value_counts())

# En caso afirmativo, muestro las columnas duplicadas:
df_duplicated_rows = df[df.duplicated()]

# Como vemos estos duplicados tienen muchos nulos por lo que aportan poca información.
display(df_duplicated_rows)

# Pero me quedo con estas filas porque tienen el campo "Country" y me permiten hacer aggregados sobre este campo.


False    1667
True        8
dtype: int64


,ID,Applies To,Country,Date Start,Date end intended,Description of measure implemented,Exceptions,Implementing City,Implementing State/Province,Keywords,Quantity,Source,Target city,Target country,Target region,Target state
16,NaN,NaN,Vietnam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,Vietnam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1322,NaN,NaN,Nepal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/2020_coronavirus...,NaN,NaN,NaN,NaN
1363,NaN,NaN,Namibia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1366,NaN,NaN,Namibia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1590,NaN,NaN,US: Massachussets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1591,NaN,NaN,US: Massachussets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1592,NaN,NaN,US: Massachussets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# ANÁLISIS DEL CAMPO "Country"

# Como vemos Estados Unidos tiene las entradas sin normalizar separadas por estados.
display(df[(df["Country"].str.contains("US:")) | (df["Country"] == "United States")]["Country"].unique())

# Aplico una función para normalizar los datos de "Country"
df['Country'] = df['Country'].apply(lambda c: "United States" if c.startswith("US:") else c)


array(['US:New Jersey', 'United States', 'US: Florida', 'US:N Carolina',
       'US: Arizona', 'US: Oregon', 'US:California', 'US: Nevada',
       'US:Idaho', 'US:Utah', 'US:Washington', 'US: S Carolina',
       'US:Georgia', 'US:Maryland', 'US: Indiana', 'US:Delaware',
       'US: Illinois', 'US: Wisconsin', 'US: Virginia', 'US: Michigan',
       'US:New York', 'US: Pennsylvania', 'US:Alabama', 'US:Alaska',
       'US: Connecticut', 'US: Massachussets', 'US:Arkansas',
       'US:Colorado', 'US:Hawaii', 'US:Iowa', 'US:Kansas', 'US:Kentucky',
       'US:Louisiana', 'US: Rhode Island'], dtype=object)

# Separar el campo Keywords

In [9]:
# Separo cada elemento que hay en Keywords (que originalmente están separados por comas)
df_clean = df.set_index(df.columns.drop('Keywords',1).tolist())['Keywords'].str.split(',', expand=True).stack().reset_index().rename(columns={0:'Keywords'})


In [10]:
# Como podemos ver en este ejemplo, el campo "Keywords" se ha separado correctamente
display(df[(df['Country'] == 'Germany') & (df['Date Start'] == "Mar 13, 2020")].head()[['Country', 'Date Start', 'Keywords']])
display(df_clean[(df_clean['Country'] == 'Germany') & (df_clean['Date Start'] == "Mar 13, 2020")].head()[['Country', 'Date Start', 'Keywords']])


,Country,Date Start,Keywords
544,Germany,"Mar 13, 2020","nursery school closure, school closure"
742,Germany,"Mar 13, 2020",economic stimulus


,Country,Date Start,Keywords
847,Germany,"Mar 13, 2020",nursery school closure
848,Germany,"Mar 13, 2020",school closure
1098,Germany,"Mar 13, 2020",economic stimulus


In [11]:
# Al final me quedo con estas columnas solo en el dataframe limpio:
df_clean = df_clean[["Country", "Date Start", "Date end intended", "Keywords"]]
display(df_clean.head())

,Country,Date Start,Date end intended,Keywords
0,Austria,"Mar 16, 2020",NaN,university closure
1,Germany,"Feb 01, 2020",NaN,international traveller quarantine - risk coun...
2,Germany,"Feb 01, 2020",NaN,traveller testing
3,United Kingdom,"Mar 20, 2020",NaN,nursery school closure
4,United Kingdom,"Mar 20, 2020",NaN,school closure


In [16]:
# Voy a normalizar algunas de las entradas de Keywords

# - Quito los espacios blancos a derecha e izquierda de las Keywords
df_clean['Keywords'] = df_clean['Keywords'].apply(lambda s: s.strip())

def normalize_keywords(keyword):
    if re.match(r"(stimulus|financial|economic)", keyword):
        return "economic stimulus"
    if re.match(r"international traveller screening.*", keyword):
        return "international traveller screening"
    if keyword in ['traveller testing', 'test travellers']:
        return "international traveller screening"
    if re.match(r"international traveller quarantine.*", keyword):
        return "international traveller quarantine"
    if re.match(r"(international|outbound) travel[^\s]* ban.*", keyword):
        return "international travel ban"
    if re.match(r"(nursery )?(school|university) closure.*", keyword):
        return "school or university closure"
    if keyword in ["public announcement", "blanket announcement", "general advice", "blanket text messaging", "information sms", "coronavirus education activities", "activism for stricter measures", "health declaration system"]:
        return "public announcement"
    if re.match(r"remote medical treatment.*", keyword):
        return "remote medical treatment"
    if re.match(r"^.*remote work.*", keyword):
        return "remote work"
    if re.match(r"^.*remote schooling.*", keyword):
        return "remote schooling"
    if re.match(r"religious activity.*", keyword):
        return "religious activity limitation"
    if re.match(r"social distancing.*", keyword):
        return "social distancing"
    if re.match(r".*nonessential.*", keyword):
        return "nonessential business suspension"
        
    return keyword

# - Normalizar
df_clean['Keywords'] = df_clean['Keywords'].apply(normalize_keywords)
